In [1]:
import bagpy
from tqdm import tqdm
from typing import *
from shapely.geometry import *
from os.path import join, exists
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import geopandas as gpd

In [ ]:
worlds_df = pd.read_pickle("/home/agusmazzeo/Documents/UTN/Proyecto/worlds_df.pkl")

In [ ]:
worlds_df

In [ ]:
class BagpyDataHandler:
    
    def __init__(self, worlds_df: pd.DataFrame, outputs_dir: str):
        self.res_values = {}
        self.report = {}
        self.worlds = {}
        for index in tqdm(worlds_df[worlds_df["finished_true"]].index):
            count+=1
            result = self.get_results_from_folder(join(outputs_dir, index))
            if result is not None:
                self.res_values[index] = result
                self.worlds[index] = worlds_df.loc[index]
                self.report[index] = self.get_report_data(index, result)
        reform = {(outerKey, innerKey): values for outerKey, innerDict in self.report.items() for innerKey, values in innerDict.items()}
        self.report_df = pd.DataFrame(reform)
                
    def get_report_data(self, index: str, result: Dict[str, Any]):
        report = {}
        world_polygon = self.worlds[index].world.free_space_polygon
        if isinstance(world_polygon, MultiPolygon):
            world_polygon = world_polygon.convex_hull
        for key in result.keys():
            if key not in report.keys():
                report[key] = {}
            poly = gpd.GeoDataFrame(geometry=[world_polygon.exterior])
            report[key] = {"trajectory": poly.append(result[key]["trajectory_df"]), **self.get_exploration_data(key, index, result)}
        return report
    
    def get_exploration_data(self, key: str, index: str, result: Dict[str, Any]):
        free_area = self.worlds[index]["free_area"]
        explored_per_distance = pd.merge_asof(result[key]["explored_df"], result[key]["travelled_distance_df"], on="timestamp")
        explored_area = explored_per_distance["m_2_explored"].iloc[-1]
        exploration = {
            "explored_percentage": 100*explored_area/free_area,
            "total_time_sec": explored_per_distance.index[-1],
            "dist_explored_20%": explored_per_distance.loc[explored_per_distance["m_2_explored"]<(20*free_area/100)].iloc[-1]["cum_distance"],
            "dist_explored_50%": explored_per_distance.loc[explored_per_distance["m_2_explored"]<(50*free_area/100)].iloc[-1]["cum_distance"],
            "dist_explored_80%": explored_per_distance.loc[explored_per_distance["m_2_explored"]<(80*free_area/100)].iloc[-1]["cum_distance"],
            "dist_total": explored_per_distance.iloc[-1]["cum_distance"],
        }
        return exploration
            
    def get_results_from_folder(self, folder: str) -> Dict[str, Tuple[List[float], List[float]]]:
        try:
            utn_path = join(folder, "UTN")
            maryland_path = join(folder, "MARYLAND")
            if not exists(utn_path) or not exists(maryland_path):
                # print(f"Skipping {folder} because of not having both maryland and UTN")
                return None
            utn_bag_file = glob(join(utn_path, "*.bag"))[0]
            maryland_bag_file = glob(join(maryland_path, "*.bag"))[0]
            reader_utn = bagpy.bagreader(utn_bag_file)
            reader_maryland = bagpy.bagreader(maryland_bag_file)
            res = {"UTN": self.get_times_and_explores(reader_utn), "MARYLAND": self.get_times_and_explores(reader_maryland)}
            return res
        except Exception as ex:
            print(f"An error ocurred on {folder}: {ex}")
            return None
    
    @staticmethod
    def get_explored_df(reader: bagpy.bagreader) -> pd.DataFrame:
        explored_data =  reader.reader.read_messages("/create1/explored")
        amt_of_non_unknown = []
        timestamps = []
        for d in explored_data:
            amt_of_non_unknown.append(d.message.data)
            timestamps.append(d.timestamp.to_sec())
        amt_of_non_unknown = [0.0025 * item for item in amt_of_non_unknown]
        dic = {"timestamp": timestamps, "m_2_explored": amt_of_non_unknown}
        df = pd.DataFrame(dic)
        df["timestamp"] -= dic["timestamp"][0]
        df = df.set_index("timestamp")
        return df

    @staticmethod
    def get_travelled_distance_df(reader: bagpy.bagreader) -> pd.DataFrame:
        travelled_data =  reader.reader.read_messages("/create1/travelled_distance")
        cum_distance = []
        timestamps = []
        for d in travelled_data:
            cum_distance.append(d.message.data)
            timestamps.append(d.timestamp.to_sec())
        dic = {"timestamp": timestamps, "cum_distance": cum_distance}
        df = pd.DataFrame(dic)
        df["timestamp"] -= dic["timestamp"][0]
        df = df.set_index("timestamp")
        return df

    @staticmethod
    def get_trajectory_df(reader: bagpy.bagreader) -> gpd.GeoDataFrame:
        gts =  reader.reader.read_messages("/create1/gts")
        xs = []
        ys = []
        timestamps = []
        for d in gts:
            xs.append(d.message.pose.pose.position.x)
            ys.append(d.message.pose.pose.position.y)
            timestamps.append(d.timestamp.to_sec())
        dic = {"timestamp": timestamps, "x": xs, "y": ys}
        df = pd.DataFrame(dic)
        df["timestamp"] -= dic["timestamp"][0]
        df = df.set_index("timestamp")
        return gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.x, df.y))

    def get_times_and_explores(self, reader: bagpy.bagreader) -> Tuple[List[float], List[float]]:
        explored_df = self.get_explored_df(reader)
        travelled_distance_df = self.get_travelled_distance_df(reader)
        trajectory_df = self.get_trajectory_df(reader)
        return {"explored_df": explored_df, "travelled_distance_df":travelled_distance_df, "trajectory_df":trajectory_df}

    def plot_exp_by_dist(self, index: str):
        res = self.res_values[index]
        utn_df=  pd.merge_asof(res["UTN"]["explored_df"], res["UTN"]["travelled_distance_df"], on="timestamp")
        maryland_df=  pd.merge_asof(res["MARYLAND"]["explored_df"], res["MARYLAND"]["travelled_distance_df"], on="timestamp")
        utn_df = utn_df.dropna()
        maryland_df = maryland_df.dropna()

        plt.plot( utn_df["timestamp"], utn_df["m_2_explored"], label="UTN")
        plt.plot( maryland_df["timestamp"], maryland_df["m_2_explored"], label="MARYLAND")
        plt.ylabel("Explored area [m2]")
        plt.xlabel("Distance travelled [m]")

        plt.legend()
    
    def plot_trajectory(self, index: str):
        report = self.report[index]
        report["UTN"]["trajectory"].buffer(0.2).plot()
        report["MARYLAND"]["trajectory"].buffer(0.2).plot()
        

In [ ]:
BASE_DIR = "/home/agusmazzeo/Documents/UTN/Proyecto/outputs"

In [ ]:
bdh = BagpyDataHandler(worlds_df, BASE_DIR) 

In [ ]:
pd.DataFrame(bdh.report['evaluation_3']).T

In [ ]:
reform = {(outerKey, innerKey): values for outerKey, innerDict in bdh.report.items() for innerKey, values in innerDict.items()}
pd.DataFrame(reform).T.loc[('evaluation_1','UTN')]